In [ ]:
import bs4
import requests
import json
import pandas as pd
from pypinyin import pinyin, lazy_pinyin, Style
import collections

In [ ]:
cookies = json.load(open('cookies.json'))
cookies

In [ ]:
res = []

In [ ]:
k = 0
flag = True

while flag:
    k += 1
    url = f"http://wjpython.openjudge.cn/admin/members/?page={k}"
    print(url)
    result = requests.get(
        url=url,
        cookies=cookies
    )
    bs4_obj = bs4.BeautifulSoup(result.text, "html.parser")
    lst = bs4_obj.find_all("input", {"name": "users[]", "type": "checkbox"})
    for tr in bs4_obj.find_all("tr")[1:-1]:
        ttt = tr.find_all("td")
        userid = int(next(ttt[0].children).attrs['value'])
        username = ttt[1].text
        year, month, _ = ttt[-1].text.split('-')
        year = int(year)
        month = int(month)
        if year >= 2021 and month >= 8:
            res.append((userid, username))
        else:
            flag = False
            break
    if not flag:
        break

In [ ]:
res

In [ ]:
df = pd.read_excel('选课名单.xls')
df.insert(0, 'userid', pd.NA)
df.insert(1, 'username', pd.NA)
df

In [ ]:
for line in df.iterrows():
    xuehao = line[1]['学号']
    xingming = line[1]['姓名']
    for userid, username in res:
        if xingming in username \
            or str(xuehao) in username \
            or (username in xingming and len(username) >= 2) \
            or "".join(lazy_pinyin(xingming)) in username.lower():
                df.loc[line[0], 'userid'] = userid
                df.loc[line[0], 'username'] = username
                break
    else:
        print(xingming, xuehao)

In [ ]:
dup = [item for item, count in collections.Counter(df['userid']).items() if count > 1]
print(dup)
assert len(dup) == 0

In [ ]:
to_save = df[['userid', 'username', '学号', '姓名']]

In [ ]:
to_save.to_csv('userid_studentid.csv', index=False)